# Lab 5: Ensemble Methods: Bagging, Boosting, Random Forests

In this lab you will get familiar with ensemble methods. We will cover bagging, boosting (AdaBoost) and random forests in the exercises in this lab. Please refer to 
http://scikit-learn.org/stable/modules/ensemble.html#adaboost for an introduction to these methods and some example usages.

# 1) Data Import and cleaning
We will use the same data as in the previous two labs (Kaggle KDD Cup 2014) so you can use the same cleaned data for
this lab session. In order to save some time you can also use the following parts taken from the solution to the
previous lab and modify as you need.

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

In [23]:
# Load data
proj = pd.read_csv('data/projects.csv')
outcomes = pd.read_csv('data/outcomes.csv')

In [24]:
# Join on project id
all_data = pd.merge(proj, outcomes, how='inner', left_on='projectid', right_on='projectid')
y_values = all_data['is_exciting'].astype(np.bool)
all_x_values = all_data[proj.columns]

# or use join:
# temp = proj.set_index('projectid').join(outcomes.set_index('projectid'))

In [29]:
# check out the columns
# all_x_values.columns

In [30]:
# pick the features you want to use
features = []
x_values = all_x_values[features]

In [31]:
# check how data looks
# x_values.head()

In [32]:
def convert_bools(df, bool_cols):
    #bool_cols = []
    for col in bool_cols:
        df[col] = df[col].astype(np.bool)
        # Alternatively
        # df[col] = df[col].apply(np.bool)

# apply it to the dataframe
bool_cols = ['eligible_double_your_impact_match']
convert_bools(x_values, bool_cols)

/Users/Samira/envipython/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [37]:
# check the data
# x_values

In [34]:
def binarize_categories(df, cat_cols, drop=True):
    '''
    df: a pandas dataframe
    cat_cols: list of column names to generate indicator columns for
    drop: a bool. If true, drop the original category columns
    Returns: the modified dataframe
    '''
    for col in cat_cols:
        binary_cols = pd.get_dummies(df[col], col)
        df = pd.merge(df, binary_cols, left_index=True, right_index=True, how='inner')
    if drop:
        df.drop(cat_cols, inplace=True, axis=1)
    return df

In [35]:
# apply it to the real dataframe
cat_cols = []
cleaned_x_values = binarize_categories(x_values, cat_cols, drop=True)

In [36]:
# check the data
# cleaned_x_values.head()

In [38]:
# drop rows
x_values = x_values.dropna()

In [16]:
# 80/20 train test split. But you can tweak the test size
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_values, y_values, test_size=0.20, random_state=17)

ValueError: Found input variables with inconsistent numbers of samples: [619159, 619326]

# 2) Bagging
You can use bagging to make a stronger estimator from simple base estimators. Bagging merges independent estimators which are made using different random subsets of the training samples. Refer to 

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html#r154 

to check the parameters. This function provides more than just bagging, for instance in addition to you can also take random subsets of the features (max_features). In order to implement bagging, you need to keep all the features but use random subsets of the samples. You can use the bootstrap parameter to specify that your samples are drawn with replacement. Use n_estimators and max_samples to specify the number of the estimators you want to use and the number of samples you want to use for each of them.

##  Bagging with logistic regression
Use logistic regression as  your base classifier. To keep it simple use l2 norm and C = 1. Since the data is very unbalanced regarding to the number of True and False samples, use the class_weight parameter to specify how much the model should prefer correctly classifying one class over another.

Define your classifier. Use fit and score functions to fit your model and compute the score.

###What happens when you try different bagging parameters?
Try n_estimator = { 5 ,10, 20 } and max_samples = { 0.35, 0.5, 0.65 } and report the results.


# 3) Boosting - AdaBoost

Another approach for making stronger estimators from the basic ones is boosting. In contrast to bagging, boosting makes a strong classifier by adding the features one by ones based on the predictive power. At each step of boosting training samples are re-weighted to give a higher weight to the ones which were wrongly classified and direct the algorithm to choose features which are useful for classifying those samples.

In this part you will make a classifier using AdaBoost which as a popular boosting algorithm. Refer to

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier

for the specifications. The default estimator is decision tree classifier but you can use any estimator of your choice as far as it has the conditions of the base estimator for AdaBoost. For instance since AdaBoost changes the weights of the samples, your base classifier should support this propoerty. Here again, you need to specify the number of simple classifiers by n_estimators.

#### Make sure you have enough features

For using AdaBoost since the base classifiers added at each step are made by taking new features, you need to make sure that you have enough variables to make the simple classifiers. So, if you took very few variables from your data to begin with, you will need to add other features for this part.


## AdaBoost with decision tree of depth one
Use decision tree of depth one as your base classifier. For Adaboost parameters use n_estimators = 20.  Again use the class_weight parameter for your decision tree classifier to deal with the unbalanced data. You may use the 'balanced' option.

Define your classifier. Use fit and score functions to fit your model and compute the score.

### What happens when you decrease or increase the number of your estimators?

Try using n_estimators = { 10, 20, 40} and report the results.

# 4) Random Forests

Another approach for making stronger estimators from the basic ones is using random forests which provides a strong etimator built from a number of decision tree estimators. Each individual decision tree is made by using a random subset of the features. In addidion to that the training samples used for each tree are also random bootstrap samples from the training set (of the same size).

In this part you will make a random forest classifier. Refer to

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

for the specifications and the parameters. You can use n_estimators and max_features to specify the number of the estimators and the number of features you want to use for building each tree. 



## Exeriment with Random Forest 

Use n_estimator = 10 and max_features=sqrt(n_features). You may use max_depth=20 in combination with min_samples_split=1 to stop the trees from growing too deep.

Define your classifier. Use fit and score functions to fit your model and compute the score.


### What happens when you use other parameters?
Use different values for n_estimator and max_depth. Report and compare your results.